In [1]:
import sys, os
sys.path.append('..')
from utils_ft import get_model_path, get_output_dir, setup_env, get_root_path
from scripts.train_pytorch import train_pytorch

# 1. Configuration
# Toggle between "llama" and "qwen" as needed
#MODEL_TYPE = "llama" 
MODEL_TYPE = "qwen"

hf_token = setup_env()
model_id = get_model_path(MODEL_TYPE)

# 2. Path Setup
# We point to the FOLDER 'data', which now contains train.jsonl and valid.jsonl
dataset_dir = os.path.join(get_root_path(), "Finetuning", "data")
output_dir = get_output_dir(MODEL_TYPE, "pytorch")

# 3. Execution
# Ensure the script receives the directory path, not a specific file
train_pytorch(
    model_id=model_id, 
    dataset_dir=dataset_dir, 
    output_dir=output_dir, 
    hf_token=hf_token
)

--- Initializing PyTorch Training ---
--- Device: MPS ---


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

The tokenizer you are loading from '/Users/arjandeweerd/AI_Projects/Ass3_LLM_ASP/SchASPLM-Shamantics/local_models/Qwen/Qwen2.5-7B-Instruct-bfloat16' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Tokenizing train dataset:   0%|          | 0/43 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/43 [00:00<?, ? examples/s]

...

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Starting training process...


/opt/homebrew/Caskroom/miniconda/base/envs/schasplm/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,1.882234,1.475956,1.227683,30238.000000,0.693741
2,1.543730,1.255234,1.273990,60476.000000,0.721265
3,1.390804,1.119386,1.195052,90714.000000,0.745829
4,1.292870,1.020619,1.043580,120952.000000,0.766574
5,1.050759,0.956215,0.911583,151190.000000,0.777007
6,1.007475,0.919510,0.843373,181428.000000,0.785566
7,0.921547,0.897240,0.811306,211666.000000,0.789437
8,0.939486,0.885992,0.799605,241904.000000,0.792611
9,0.863553,0.877846,0.796375,272142.000000,0.793922
10,0.869738,0.877709,0.795267,302380.000000,0.792611


Saving merged model to: ./local_models/ft_qwen_pytorch


In [5]:
import json
import pandas as pd
def plot_training_results(log_dir):
    report_path = os.path.join(log_dir, "report.json")
    
    if not os.path.exists(report_path):
        print(f"Error: report.json not found in ./{to_relative(log_dir)}")
        return

    with open(report_path, "r") as f:
        log_data = json.load(f)

    # Convert to DataFrame for easier manipulation
    df = pd.DataFrame(log_data)

    # Detect format: MLX uses 'train_loss', HF uses 'loss'
    train_col = 'train_loss' if 'train_loss' in df.columns else 'loss'
    val_col = 'val_loss' if 'val_loss' in df.columns else 'eval_loss'
    iter_col = 'iteration' if 'iteration' in df.columns else 'step'

    # Filter out empty rows for specific metrics
    train_df = df[df[train_col].notna()]
    val_df = df[df[val_col].notna()]

    plt.figure(figsize=(10, 5))
    
    if not train_df.empty:
        plt.plot(train_df[iter_col], train_df[train_col], label='Training Loss', alpha=0.7)
    
    if not val_df.empty:
        plt.plot(val_df[iter_col], val_df[val_col], label='Validation Loss', color='red', marker='x')

    plt.title('ASP Autoformalization Fine-tuning Progress')
    plt.xlabel('Iterations / Steps')
    plt.ylabel('Loss')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

# Run the visual inspection
# For MLX, point to the adapter folder. For PyTorch, point to the output folder.
plot_training_results(output_dir)

NameError: name 'plt' is not defined